In [1]:
from flask_login import current_user
import visual_library_plos as vl
import simplejson
from urllib.parse import urlencode
from urllib.request import urlopen
from io import StringIO
from contextlib import closing
from flask_session import Session
from flask import (
    render_template,
    request,
    redirect,
    url_for,
    send_from_directory,
    g,
    flash,
    session,
    jsonify,
    Response,
    json,
)
from sklearn import preprocessing
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix
from scipy.spatial.distance import pdist, squareform
import pandas as pd
import numpy as np
from app import app, tokenize, exclude, stopwords, MINDF
from db.db_get_data import get_dc_data

# import argparse
# import sys
# import operator
import math

# import gzip
# import random
import os
import time

time.clock = time.time
from datetime import datetime
from datetime import timedelta
from collections import defaultdict
import json

import pysolr
from elasticsearch import Elasticsearch

es = Elasticsearch()

Reading stopwords...


DEBUG:git.cmd:Popen(['git', 'version'], cwd=/Users/aravind/Documents/LAIR Hub/CODE_v2/pattie_humanities, stdin=None, shell=False, universal_newlines=False)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/Users/aravind/Documents/LAIR Hub/CODE_v2/pattie_humanities, stdin=None, shell=False, universal_newlines=False)


/Users/aravind/Documents/LAIR Hub/CODE_v2/pattie_humanities


In [1]:
!pwd

/Users/aravind/Documents/LAIR Hub/CODE_v2/pattie_humanities


In [2]:
def cluster(raw_query_inp, num_cls_inp):
    print("cluster")
    error = None
    dataset = "PubMedAPI"

    field = "All fields"
    raw_query = raw_query_inp
    entity = "keywords"
    q = raw_query
    # print(q)
    num_cls = num_cls_inp

    if dataset == "Placeholder":
        return redirect(url_for("home"))
    
    # dynamic dataset
    elif dataset == "PubMedAPI":
        print("PubMedAPI in routes.py")
        import cluster_pymedAPI
        import es_search

        # print("start_date")
        # print(start_date)
        num_cls = int(num_cls)
        # try:
        # data, error = es_search.retrieve(q, entity)
        data, error = cluster_pymedAPI.retrieve(q)
        # data, error = get_dc_data(q)
        # http://localhost:5000/cluster?dataset_opt=PubMedAPI&query=Javed Mostafa
        # http://localhost:5000/cluster?dataset_opt=PLOS&query=digital%20health
        # http://localhost:5000/cluster?dataset_opt=NewsAPI&query=digital%20health
        print(data, error)
        print(type(data), data.columns.tolist())
        # except:
        #     flash("The National Library of Medicine Entrez API is experiencing technical issues. We're sorry for this inconvenience.")
        #     return redirect(url_for('home'))

        if error == None:
            if entity == "genes":
                doc_term_mat, df, w2id, bibs = vl.read_df_genes(
                    session["gene_set"], data, dataset, stopwords
                )
                # print('-----------df------------')
                # print(df)
            elif entity == "authors":
                # doc_term_mat, df, w2id, bibs = vl.read_df_authors(
                #     data, dataset, stopwords)
                (
                    orig_doc_term_mat,
                    orig_df,
                    df,
                    dfr,
                    keywords,
                    cluster_centers,
                    cluster_desc,
                    coordinates,
                    id2members,
                    bibs,
                    org_ids,
                    author_names,
                    author_docs,
                ) = vl.process_authors_4(data, dataset, stopwords)
            else:
                doc_term_mat, df, w2id, bibs = vl.read_df(data, dataset, stopwords)
        else:
            # flash(error)
            return redirect(url_for("home"))

    # uploaded dataset
    else:
        # num_cls = 10
        uploaded_file = dataset
        # path = UPLOAD_FOLDER  # needs to be changed
        # print(path)
        doc_term_mat, df, w2id, bibs = vl.read_plaintext(
            path, uploaded_file, q, dataset, stopwords
        )

    # print("doc_term_mat")
    # print(doc_term_mat)
    # print(df)

    if dataset != "Experts" and len(bibs) < 10:
        #       flash('No clusters found. Broaden your search.')
        flash(
            "Please upload larger files or search for more relevant keywords to get enough results."
        )
        #        return redirect(url_for('home'))
        return redirect(request.referrer)

    if entity != "authors" and dataset != "Experts":
        # Remove terms whose df is lower than mindf
        if MINDF > 0:
            inf = []
            for w in df:
                if df[w] <= MINDF:
                    inf.append(w)
            for w in inf:
                del df[w]

        # Save org data
        orig_doc_term_mat = doc_term_mat
        orig_df = df

        # Compute tfidf and find key terms
        # print("Computing TFIDF and finding key terms...")
        if dataset == "NYTIMES" or dataset == "PLOS" or dataset == "DIABETES":
            doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=10)
        else:
            doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=30)

        # Sort and output results (discovered keywords)
        keywords = vl.output_keywords(len(doc_term_mat), dfr, df, p_docs=1.0)
        # print('Keywords...')
        # print(keywords)

        # Create new matrix with the keywords
        doc_term_mat, org_ids = vl.update(doc_term_mat, keywords)

        # Convert to sparse matrix
        doc_term_mat = vl.convert_sparse(doc_term_mat, keywords)

        # Clustering
        # print()
        # print("Clustering...")

        # n_components: number of dimensions for LSA
        # k: number of clusters
        # n_desc: number of keywords (desc) for each cluster

        if dataset == "PLOS":
            id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
                doc_term_mat, keywords, org_ids, n_components=50, k=num_cls, n_desc=25
            )
        else:
            id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
                doc_term_mat, keywords, org_ids, n_components=20, k=num_cls, n_desc=15
            )

        if error != None:  # needs to be changed
            print(error)
            flash(
                "Sorry, the entity you've chosen does not generate meaningful clusters for current query. Please try other entities or queries. "
            )
            return redirect(request.referrer)

    # get cluster colors
    """
    # center is (.5,0)
    cc = np.array(cluster_centers)-[.5,0]
    hue = ((np.arctan2(cc[:,1],cc[:,0])/np.pi+1)*180*2).\
        astype(int).tolist()
    satr = (np.linalg.norm(cc, axis=1)/math.sqrt(5)*2).tolist()
    """
    # center is (.5,.5)
    cc = np.array(cluster_centers) - 0.5
    hue = ((np.arctan2(cc[:, 1], cc[:, 0]) / np.pi + 1) * 180).astype(int).tolist()
    satr = (np.linalg.norm(cc, axis=1) / math.sqrt(2) * 2).tolist()
    val = [0.8] * num_cls

    # Create data to pass to result.html
    df_new = dict()
    dfr_new = dict()
    for w in keywords:
        df_new[w] = df[w]
        dfr_new[w] = dfr[w]

    # create adjacency matrix that will be used for network edges
    centroid_matrix = pd.DataFrame.from_records(cluster_centers)
    centroid_distances = pd.DataFrame(
        squareform(pdist(centroid_matrix, metric="euclidean")),
        columns=list(id2members.keys()),
        index=list(id2members.keys()),
    ).to_dict()

    id2freq = {x: len(id2members[x]) for x in id2members}

    # create network data structure
    # print('\nEUCLIDEAN DISTANCES\n----------')
    edges = []
    for i in id2members:
        for k, v in centroid_distances[i].items():
            # print(i, ' to ', k, ' = ', v)
            if i != k:
                if v > 0.75:
                    v = "Distant"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.75 and v > 0.50:
                    v = "Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.50:
                    v = "Very Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)

                # elif v >= 0.50:
                #     v = "Not Similar"
                # edge = {"clusterID":i,"source":cluster_centers[i],"target":cluster_centers[k],"distance":v}
                # edges.append(edge)
    # print('\nBIBLIOGRAPHY\n----------')
    sources = []
    id2meshTerms = {}
    if dataset == "PubMedAPI":
        from collections import Counter

        for cluster_id in sorted(id2members.keys()):
            concepts = cluster_desc[cluster_id]
            references = []
            for bib_id in id2members[cluster_id]:
                references.append(bibs[bib_id])
            for paper in references:
                if type(paper["author"]) != str and None in paper["author"]:
                    paper["author"] = [i for i in paper["author"] if i]
            bibliography = {"concepts": concepts, "references": references}
            meshTerms = []
            for ref in references:
                meshHeadingList = ref.get("meshHeadings")  # ref["meshHeadings"]
                for mesh in meshHeadingList:
                    meshTerms.append(mesh["descriptor"])
            # print('concept: {}'.format(concepts))
            # print('meshTerms: {}'.format(Counter(meshTerms)))
            # print('\n')
            id2meshTerms[cluster_id] = str(dict(Counter(meshTerms).most_common(20)))
            # vl.genes_coverage(concepts, references)
            sources.append(bibliography)
        # print(sources)

    """
    cnt = np.unique(membership, return_counts=True)
    keys = [x for x in cnt[0]]
    values = [int(x) for x in cnt[1]]
    id2freq = dict(zip(keys, values))
    """

    """
    # Prepare bib data if num of documents is small
    if len(org_ids) > 150:
        bibs = []
    """

    # session['doc_term_mat'] = doc_term_mat  # term-doc matrix
    session = {}
    session["docs_org"] = orig_doc_term_mat  # doc-term raw freq matrix
    session["df_org"] = orig_df
    session["num_cls"] = num_cls
    session["id2members_0"] = id2members
    session["cluster_desc_0"] = cluster_desc
    session["xy_0"] = cluster_centers
    # session["state"] = state
    session["hue_0"] = hue
    session["satr_0"] = satr
    session["org_ids_0"] = org_ids
    session["bibs_0"] = bibs
    session["dataset"] = dataset
    session["edges_0"] = edges
    session["sources"] = sources
    session["id2meshTerms"] = id2meshTerms

    session["entity"] = entity
    if entity == "authors":
        session["author_names"] = author_names
        session["author_docs"] = author_docs
        
    # Convert NumPy arrays to lists
    def convert_ndarray_to_list(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, list):
            return [convert_ndarray_to_list(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: convert_ndarray_to_list(value) for key, value in obj.items()}
        else:
            return obj

    # Ensure orig_doc_term_mat is a list
    orig_doc_term_mat_list = convert_ndarray_to_list(orig_doc_term_mat)
    # Ensure orig_df is a list or dict (depending on its original type)
    orig_df_list = convert_ndarray_to_list(orig_df)
    # Ensure cluster_centers is a list
    cluster_centers_list = convert_ndarray_to_list(cluster_centers)
    # Ensure coordinates is a list
    coordinates_list = convert_ndarray_to_list(coordinates)


    sessionData = {
        "docs_org": convert_ndarray_to_list(orig_doc_term_mat),  # doc-term raw freq matrix
        "df_org": convert_ndarray_to_list(orig_df),
        "num_cls": num_cls,
        "id2members_0": convert_ndarray_to_list(id2members),
        "cluster_desc_0": convert_ndarray_to_list(cluster_desc),
        "xy_0": convert_ndarray_to_list(cluster_centers),
        # "state": state,
        "hue_0": convert_ndarray_to_list(hue),
        "satr_0": convert_ndarray_to_list(satr),
        "org_ids_0": convert_ndarray_to_list(org_ids),
        "dataset": dataset,
        "edges_0": convert_ndarray_to_list(edges),
        "sources": convert_ndarray_to_list(sources)
    }

    # session['df'] = df_new
    # session['dfr'] = dfr_new
    # session['keywords'] = keywords

    # print("bibs")
    # print(bibs)

    # output the data to terminal
    # print('\nCENTROIDS\n----------')
    # print(cluster_centers)
    # print('\nCENTROID DISTANCES\n----------')
    # print(centroid_distances)
    # print('\nCLUSTER LABELS\n----------')
    # print(cluster_desc)
    # print('\nCLUSTER ID & PUBLICATIONS\n----------')
    # print(id2members)
    # print('\nCLUSTER ID & PUBLICATION COUNT\n----------')
    # print(id2freq)
    # print('\nNETWORK\n----------')
    # print(edges)



    if entity == "authors" or entity == "experts" or entity == "topics":
        # to tell the difference between keyword search and author search so that sessionStorage won't keep old data when switching between them
        q = q + "_" + entity

    print("exit cluster")
    return {
        "sessionData": sessionData,
        "raw_query": raw_query,
        "q": q,
        "error": error,
        "keywords": keywords,
        "df": df_new,
        "dfr": dfr_new,
        "cluster_desc": cluster_desc,
        "xy": cluster_centers_list,
        "edges": edges,
        "id2freq": id2freq,
        "xy_inter": coordinates_list,
        "hue": hue,
        "satr": satr,
        "id2members": id2members,
        "sources": sources,
        "dataset": dataset,
        "num_cls": num_cls,
    }

In [3]:
cluster("Fei Yu", 10)

DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


cluster
PubMedAPI in routes.py


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


Retrieving data from PubMed


Traceback (most recent call last):
  File "/Users/aravind/Documents/LAIR Hub/CODE_v2/pattie_humanities/cluster_pymedAPI.py", line 35, in retrieve
    results = pubmed.query(query, max_results=300)
  File "/Users/aravind/anaconda3/envs/PATTIE_v2/lib/python3.10/site-packages/pymed/api.py", line 62, in query
    article_ids = self._getArticleIds(query=query, max_results=max_results)
  File "/Users/aravind/anaconda3/envs/PATTIE_v2/lib/python3.10/site-packages/pymed/api.py", line 208, in _getArticleIds
    response = self._get(url="/entrez/eutils/esearch.fcgi", parameters=parameters)
  File "/Users/aravind/anaconda3/envs/PATTIE_v2/lib/python3.10/site-packages/pymed/api.py", line 140, in _get
    response = requests.get(f"{BASE_URL}{url}", params=parameters)
  File "/Users/aravind/anaconda3/envs/PATTIE_v2/lib/python3.10/site-packages/requests/api.py", line 73, in get
    return request("get", url, params=params, **kwargs)
  File "/Users/aravind/anaconda3/envs/PATTIE_v2/lib/python3.10/site-pa

Unexpected error:  None


KeyboardInterrupt: 

In [ ]:
def cluster(raw_query_inp, num_cls_inp):
    print("cluster")
    error = None
    dataset = "PubMedAPI"

    field = "All fields"
    raw_query = raw_query_inp
    entity = "keywords"
    q = raw_query
    # print(q)
    num_cls = num_cls_inp

    if dataset == "Placeholder":
        return redirect(url_for("home"))
    
    # dynamic dataset
    elif dataset == "PubMedAPI":
        print("PubMedAPI in routes.py")
        import cluster_pymedAPI
        import es_search

        # print("start_date")
        # print(start_date)
        num_cls = int(num_cls)
        # try:
        # data, error = es_search.retrieve(q, entity)
        data, error = cluster_pymedAPI.retrieve(q)
        print(data, error)
        print(type(data), data.columns.tolist())
        if error == None:
            if entity == "authors":
                # doc_term_mat, df, w2id, bibs = vl.read_df_authors(
                #     data, dataset, stopwords)
                (
                    orig_doc_term_mat,
                    orig_df,
                    df,
                    dfr,
                    keywords,
                    cluster_centers,
                    cluster_desc,
                    coordinates,
                    id2members,
                    bibs,
                    org_ids,
                    author_names,
                    author_docs,
                ) = vl.process_authors_4(data, dataset, stopwords)
            else:
                doc_term_mat, df, w2id, bibs = vl.read_df(data, dataset, stopwords)
        else:
            # flash(error)
            return redirect(url_for("home"))

    # print("doc_term_mat")
    # print(doc_term_mat)
    # print(df)

    if dataset != "Experts" and len(bibs) < 10:
        #       flash('No clusters found. Broaden your search.')
        flash(
            "Please upload larger files or search for more relevant keywords to get enough results."
        )
        #        return redirect(url_for('home'))
        return redirect(request.referrer)

    if entity != "authors" and dataset != "Experts":
        # Remove terms whose df is lower than mindf
        if MINDF > 0:
            inf = []
            for w in df:
                if df[w] <= MINDF:
                    inf.append(w)
            for w in inf:
                del df[w]

        # Save org data
        orig_doc_term_mat = doc_term_mat
        orig_df = df

        # Compute tfidf and find key terms
        # print("Computing TFIDF and finding key terms...")
        if dataset == "NYTIMES" or dataset == "PLOS" or dataset == "DIABETES":
            doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=10)
        else:
            doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=30)

        # Sort and output results (discovered keywords)
        keywords = vl.output_keywords(len(doc_term_mat), dfr, df, p_docs=1.0)
        # print('Keywords...')
        # print(keywords)

        # Create new matrix with the keywords
        doc_term_mat, org_ids = vl.update(doc_term_mat, keywords)

        # Convert to sparse matrix
        doc_term_mat = vl.convert_sparse(doc_term_mat, keywords)

        # Clustering
        # print()
        # print("Clustering...")

        # n_components: number of dimensions for LSA
        # k: number of clusters
        # n_desc: number of keywords (desc) for each cluster

        if dataset == "PLOS":
            id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
                doc_term_mat, keywords, org_ids, n_components=50, k=num_cls, n_desc=25
            )
        else:
            id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
                doc_term_mat, keywords, org_ids, n_components=20, k=num_cls, n_desc=15
            )

        if error != None:  # needs to be changed
            print(error)
            flash(
                "Sorry, the entity you've chosen does not generate meaningful clusters for current query. Please try other entities or queries. "
            )
            return redirect(request.referrer)

    # get cluster colors
    """
    # center is (.5,0)
    cc = np.array(cluster_centers)-[.5,0]
    hue = ((np.arctan2(cc[:,1],cc[:,0])/np.pi+1)*180*2).\
        astype(int).tolist()
    satr = (np.linalg.norm(cc, axis=1)/math.sqrt(5)*2).tolist()
    """
    # center is (.5,.5)
    cc = np.array(cluster_centers) - 0.5
    hue = ((np.arctan2(cc[:, 1], cc[:, 0]) / np.pi + 1) * 180).astype(int).tolist()
    satr = (np.linalg.norm(cc, axis=1) / math.sqrt(2) * 2).tolist()
    val = [0.8] * num_cls

    # Create data to pass to result.html
    df_new = dict()
    dfr_new = dict()
    for w in keywords:
        df_new[w] = df[w]
        dfr_new[w] = dfr[w]

    # create adjacency matrix that will be used for network edges
    centroid_matrix = pd.DataFrame.from_records(cluster_centers)
    centroid_distances = pd.DataFrame(
        squareform(pdist(centroid_matrix, metric="euclidean")),
        columns=list(id2members.keys()),
        index=list(id2members.keys()),
    ).to_dict()

    id2freq = {x: len(id2members[x]) for x in id2members}

    # create network data structure
    # print('\nEUCLIDEAN DISTANCES\n----------')
    edges = []
    for i in id2members:
        for k, v in centroid_distances[i].items():
            # print(i, ' to ', k, ' = ', v)
            if i != k:
                if v > 0.75:
                    v = "Distant"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.75 and v > 0.50:
                    v = "Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.50:
                    v = "Very Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)

                # elif v >= 0.50:
                #     v = "Not Similar"
                # edge = {"clusterID":i,"source":cluster_centers[i],"target":cluster_centers[k],"distance":v}
                # edges.append(edge)
    # print('\nBIBLIOGRAPHY\n----------')
    sources = []
    id2meshTerms = {}
    if dataset == "PubMedAPI":
        from collections import Counter

        for cluster_id in sorted(id2members.keys()):
            concepts = cluster_desc[cluster_id]
            references = []
            for bib_id in id2members[cluster_id]:
                references.append(bibs[bib_id])
            for paper in references:
                if type(paper["author"]) != str and None in paper["author"]:
                    paper["author"] = [i for i in paper["author"] if i]
            bibliography = {"concepts": concepts, "references": references}
            meshTerms = []
            for ref in references:
                meshHeadingList = ref.get("meshHeadings")  # ref["meshHeadings"]
                for mesh in meshHeadingList:
                    meshTerms.append(mesh["descriptor"])
            # print('concept: {}'.format(concepts))
            # print('meshTerms: {}'.format(Counter(meshTerms)))
            # print('\n')
            id2meshTerms[cluster_id] = str(dict(Counter(meshTerms).most_common(20)))
            # vl.genes_coverage(concepts, references)
            sources.append(bibliography)
        # print(sources)

    """
    cnt = np.unique(membership, return_counts=True)
    keys = [x for x in cnt[0]]
    values = [int(x) for x in cnt[1]]
    id2freq = dict(zip(keys, values))
    """

    """
    # Prepare bib data if num of documents is small
    if len(org_ids) > 150:
        bibs = []
    """

    # session['doc_term_mat'] = doc_term_mat  # term-doc matrix
    session = {}
    session["docs_org"] = orig_doc_term_mat  # doc-term raw freq matrix
    session["df_org"] = orig_df
    session["num_cls"] = num_cls
    session["id2members_0"] = id2members
    session["cluster_desc_0"] = cluster_desc
    session["xy_0"] = cluster_centers
    # session["state"] = state
    session["hue_0"] = hue
    session["satr_0"] = satr
    session["org_ids_0"] = org_ids
    session["bibs_0"] = bibs
    session["dataset"] = dataset
    session["edges_0"] = edges
    session["sources"] = sources
    session["id2meshTerms"] = id2meshTerms

    session["entity"] = entity
    if entity == "authors":
        session["author_names"] = author_names
        session["author_docs"] = author_docs
        
    # Convert NumPy arrays to lists
    def convert_ndarray_to_list(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, list):
            return [convert_ndarray_to_list(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: convert_ndarray_to_list(value) for key, value in obj.items()}
        else:
            return obj

    # Ensure orig_doc_term_mat is a list
    orig_doc_term_mat_list = convert_ndarray_to_list(orig_doc_term_mat)
    # Ensure orig_df is a list or dict (depending on its original type)
    orig_df_list = convert_ndarray_to_list(orig_df)
    # Ensure cluster_centers is a list
    cluster_centers_list = convert_ndarray_to_list(cluster_centers)
    # Ensure coordinates is a list
    coordinates_list = convert_ndarray_to_list(coordinates)


    sessionData = {
        "docs_org": convert_ndarray_to_list(orig_doc_term_mat),  # doc-term raw freq matrix
        "df_org": convert_ndarray_to_list(orig_df),
        "num_cls": num_cls,
        "id2members_0": convert_ndarray_to_list(id2members),
        "cluster_desc_0": convert_ndarray_to_list(cluster_desc),
        "xy_0": convert_ndarray_to_list(cluster_centers),
        # "state": state,
        "hue_0": convert_ndarray_to_list(hue),
        "satr_0": convert_ndarray_to_list(satr),
        "org_ids_0": convert_ndarray_to_list(org_ids),
        "dataset": dataset,
        "edges_0": convert_ndarray_to_list(edges),
        "sources": convert_ndarray_to_list(sources)
    }

    # session['df'] = df_new
    # session['dfr'] = dfr_new
    # session['keywords'] = keywords

    # print("bibs")
    # print(bibs)

    # output the data to terminal
    print('\nCENTROIDS\n----------')
    print(cluster_centers)
    print('\nCENTROID DISTANCES\n----------')
    print(centroid_distances)
    print('\nCLUSTER LABELS\n----------')
    print(cluster_desc)
    print('\nCLUSTER ID & PUBLICATIONS\n----------')
    print(id2members)
    print('\nCLUSTER ID & PUBLICATION COUNT\n----------')
    print(id2freq)
    print('\nNETWORK\n----------')
    print(edges)



    if entity == "authors" or entity == "experts" or entity == "topics":
        # to tell the difference between keyword search and author search so that sessionStorage won't keep old data when switching between them
        q = q + "_" + entity

    print("exit cluster")
    
cluster("Fei Yu", 10)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


cluster
PubMedAPI in routes.py
Retrieving data from PubMed


DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/esearch.fcgi?tool=PATTIE&email=nibras.rakib%40mail.utoronto.ca&db=pubmed&term=Fei+Yu&retmax=300&retmode=json HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?tool=PATTIE&email=nibras.rakib%40mail.utoronto.ca&db=pubmed&id=39024040&id=39018421&id=39015361&id=38992675&id=38990325&id=38984807&id=38981857&id=38978048&id=38977905&id=38975298&id=38969409&id=38953634&id=38950519&id=38949397&id=38942189&id=38941917&id=38925411&id=38914695&id=38905100&id=38901733&id=38897173&id=38890248&id=38889048&id=38878672&id=38878412&id=38875577&id=38871478&id=38860211&id=38858958&id=38858912&id=38858393&id=38852399&id=38849239&id=38839038&id=38828289&id=38827471&id=38822373&id=38820538&id=38817980&id=38813167&id=38802944&id=38789713&id=38782278&id=38781292&id=38766678

0      [Kai-Wen Tong, Fei Yu, Hao Wang, Kang Huang, Z...
1      [Qian Niu, Tao-Yuan Yu, Jing-Wen Shi, Qing Hua...
2      [Zhegang Zhou, Longbiao Yu, Fanbin Meng, Jingj...
3      [Yang Ji, Chuangye Ni, Yanjun Shen, Zhenggang ...
4      [Fei Yu, Mingguang Yang, Cheng He, Yanli Yang,...
                             ...                        
295    [Yao Cai, Fei Yu, Manish Kumar, Roderick Gladn...
296    [Shumin Xie, Runyao Liu, Huiling Zhang, Fei Yu...
297    [Bo Zhou, Haotian Qin, Yicong Huang, Quanzhen ...
298    [Xiaoju Shen, Xiaocheng Mo, Weidan Tan, Xiaoxi...
299     [Yewen Wang, Chunzhi Gong, Fei Yu, Quanyi Zhang]
Name: authors, Length: 300, dtype: object <class 'pandas.core.series.Series'>
Going to return df
                                               authors  \
0    [Kai-Wen Tong, Fei Yu, Hao Wang, Kang Huang, Z...   
1    [Qian Niu, Tao-Yuan Yu, Jing-Wen Shi, Qing Hua...   
2    [Zhegang Zhou, Longbiao Yu, Fanbin Meng, Jingj...   
3    [Yang Ji, Chuangye Ni, Yanjun Shen, Zhe

/Users/aravind/anaconda3/envs/PATTIE_v2/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
